### Day 13: 更多list小练习，结合类和魔法方法

今天，让我们来创建一个像列表一样的类，让这个类和列表有一样的特性。哈哈哈这样Sarah又可以学到列表，又可以学到类和魔法方法。我是不是天才

emmmm 好吧，好像并不是。Sarah可能会问：那我在类里面套一个list不就行了吗？为什么要这么麻烦？

emmmm 我想到，Sarah总是觉得Python简单得让人感觉很难受，因为好多高级的方法把底层的东西都隐藏掉了。那么让我们只在类里面使用一个最基本的C语言数组。

Python的列表和C语言的数组的区别一览 (我们将要实现的部分)：

1. **边界检查**: 访问列表时，如果索引超出了范围，我们会像Python的列表一样，抛出一个`IndexError`异常，而非SegFault。

2. **负索引**: 我们可以使用负数作为索引来从后往前访问列表的元素，例如 `my_list[-1]` 访问最后一个元素。

3. **打印输出**: 我们会实现 `__repr__` 和 `__str__` 方法，这样就可以用 `print()` 函数来清晰地看到我们列表的内容了。

4. **多种类型**: 我们的列表可以存储多种类型的元素，包括整数、字符串、对象等，然而C语言的数组通常只能存储同一类型的元素。

In [10]:
# Note: Python不支持指针功能，所以我们使用`id()`函数来somehow模拟指针。
# 但是这个只是玩玩，演示用途哈哈哈
# 另外这个函数是Gemini写的，我也不知道什么意思，就这样用吧😓

import gc

def get_object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return obj
    return None

def is_passed_by_value(obj):
    """
    判断对象是否是通过值传递的。
    :param obj: 需要检查的对象
    :return:  True 如果对象是通过值传递的，False 如果对象是通过引用传递的。
    """
    # 获取对象的ID
    object_id = id(obj)
    
    # 通过ID获取对象
    retrieved_obj = get_object_by_id(object_id)
    
    # 如果能通过ID找到对象，说明它是通过引用传递的
    if retrieved_obj is not None:
        return False
    
    # 如果找不到，说明它不是通过引用传递的
    return True

In [34]:
class MyList:
    def __init__(self):
        # 方便起见，我们只允许创建空的列表，然后往里面添加东西

        # 这是内部管理的空列表，固定长度，只能存放一种数据，并且没有花里胡哨的方法，只存放id
        self._list = [None] * 1000 # 预设长度为1000
        self._len =  0  # 当前列表长度

    def __getitem__(self, index):
        # 允许通过索引访问元素，附带边界检查
        # if index == -1
        if index < 0 or index >= self._len:
            raise IndexError
        _id = self._list[index]
        return get_object_by_id(_id)
            
    def __setitem__(self, index, value):
        # 允许通过索引设置元素，附带边界检查
        if index < 0 or index >= self._len:
            raise IndexError
        self._list[index] = id(value)
    
    def __contains__(self, value):
        # 允许检查元素是否在列表中，在 xx in xx 语句中使用
        for i in range(self._len):
            _id = self._list[i]
            if get_object_by_id(_id) == value:
                return True
        return False
    
    def append(self, value):
        # 允许添加元素到列表末尾
        if self._len >= 1000:
            raise ValueError("the list is full")
        self._list[self._len] = id(value)
        self._len += 1

    def remove(self, value):
        # 允许删除列表中的元素
        for i in range(self._len):
            if get_object_by_id(self._list[i]) == value:
                for j in range(i, self._len):
                    next_item_id= self._list[j + 1]
                    self._list[j] = next_item_id
                self._len -= 1
                return
        raise ValueError("not found")
    
    def insert(self, index, value):
        # 允许在指定位置插入元素
        if index < 0 and index >= self._len:
            raise IndexError
        # for i in range(self._len - index):
        #     j = self._len - i
        #     self._list[j] = self._list[j - 1]
        if self._len >= 1000:
            raise IndexError
        for i in range(self._len, index, -1):
            self._list[i] = self._list[i - 1]
        self._list[index] = id(value)
        self._len += 1
    
    def __str__(self):
        # 返回列表的字符串表示，可能在print时使用
        s = "["
        for i in range(self._len):
            s += repr(get_object_by_id(self._list[i]))
            if i != self._len - 1:
                s += ", "
        s += "]"
        return s

    def __repr__(self):
        # 返回列表的字符串表示
        return self.__str__()

另：我叫Gemini生成了一个Python列表的各个操作的时间复杂度表格，供参考：

| 函数/操作          | 平均时间复杂度 | 最坏时间复杂度 | 备注                                     |
| :----------------- | :------------- | :------------- | :--------------------------------------- |
| `list[index]` (访问) | O(1)           | O(1)           | 通过索引直接访问元素                     |
| `len(list)`        | O(1)           | O(1)           | 获取列表长度                             |
| `list.append(item)`| O(1)           | O(N)           | 在列表末尾添加元素，扩容时为O(N)         |
| `list.pop()`       | O(1)           | O(1)           | 移除并返回列表末尾元素                   |
| `list.insert(index, item)` | O(N)           | O(N)           | 在指定位置插入元素，需要移动后续元素     |
| `list.pop(index)`  | O(N)           | O(N)           | 移除并返回指定位置元素，需要移动后续元素 |
| `value in list` (查找) | O(N)           | O(N)           | 顺序查找元素                             |
| `list.sort()`      | O(N log N)     | O(N log N)     | 原地排序，基于 Timsort 算法              |
| `list.reverse()`   | O(N)           | O(N)           | 原地反转列表                             |

下面，让我们测试你写的类吧！Note: 不要传递pass by value的参数吧，如果传递了这样的值，就让它报错吧

In [35]:
class testObject:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"testObject({self.name})"

class testObject2:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"testObject2({self.name})"

In [36]:
obj1 = testObject("obj1")
obj2 = testObject2("obj2")
obj3 = testObject("obj3")

In [37]:
l1 =  []  # 重置 l1
l2 = MyList()  # 重置 l2
l1, l2  # 这回调用 `__repr__` 还是 `__str__` 我忘了

([], [])

In [38]:
# append
l1.append(obj1)
l2.append(obj1)
l1, l2

([testObject(obj1)], [testObject(obj1)])

In [39]:
# __getitem__
l2[0]

testObject(obj1)

In [40]:
# __setitem__
l2[0] = obj2  # 修改 l2 中的第一个元素
l1, l2

([testObject(obj1)], [testObject2(obj2)])

In [41]:
# remove
l1.remove(obj1)  # 从 l1 中删除 obj1
l2.remove(obj2)  # 从 l2 中删除 obj2
l1, l2

([], [])

In [42]:
# contains
l1.append(obj1)  # 再次添加 obj1 到 l1
l2.append(obj2)  # 再次添加 obj2 到 l2
obj1 in l1, obj2 in l2

(True, True)

In [43]:
# insert
l2.insert(1, obj3)
l2

[testObject2(obj2), testObject(obj3)]